In [12]:
from all_functions import *
from sklearn.metrics import mean_absolute_percentage_error as mape
from aeon.visualisation import plot_series
import pywt
from scipy import signal
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField
from pyts.image import RecurrencePlot
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV, HuberRegressor, Lasso, LassoLarsCV
from aeon.transformations.collection.convolution_based import (
    MiniRocket,
    Rocket,
    MiniRocketMultivariateVariable,
)
import warnings

def rolling_window_image_concat(series, window, rep1, rep2, wavelet1, wavelet2, level1, level2):
  data = []
  for i in range(len(series)-window):
    example = np.array(series[i:i+window+1])
    target = example[-1]

    features = np.delete(example, -1)
    features_norm = znorm(features)
    
    target_norm = znorm_by(target, features)

    rep_features1 = transform_series(features_norm, rep1, wavelet1, level1)
    rep_features2 = transform_series(features_norm, rep2, wavelet2, level2)

    reps = np.concatenate((rep_features1.flatten(),rep_features2.flatten()))
    feat_target = np.concatenate((reps, [target_norm]))
    data.append(feat_target)
  df = pd.DataFrame(data)
  return df

In [13]:
representation = "SWT"
rep1 = "DWT"
rep2 = "SWT"
window = 36
horizon = 12
transform = "deseasonal"
derivado = "gasolinac"
estado = "SP"
series = read_series(f'../datasets/venda/mensal/uf/{derivado}/mensal_{estado.lower()}_{derivado}.csv')
train, test = train_test_stats(series, horizon)
train_tf = transform_train(train, format="normal")
# wavelet = "db4"
level = 2
data = rolling_window_image_concat(pd.concat([train_tf, pd.Series([0,0,0,0,0,0,0,0,0,0,0,0], index=test.index)]), window, rep1, rep2, "bior2.2", "bior3.3", 8, 2)
X_train, X_test, y_train, _ = train_test_split(data, horizon)
rg = RidgeCV(alphas=np.logspace(-3, 3, 10))
rg.fit(X_train, y_train)
    
predictions = recursive_multistep_forecasting(X_test, rg, horizon)
preds = pd.Series(predictions, index=test.index)
preds_concat = reverse_regressors(train, preds, format=transform)
preds_concat

timestamp
2023-03    900752.992853
2023-04    857056.002547
2023-05    866154.194099
2023-06    788434.300048
2023-07    818938.844821
2023-08    842792.925431
2023-09    843001.053706
2023-10    859150.711532
2023-11    849777.551065
2023-12    959250.325640
2024-01    810175.287448
2024-02    810572.379248
Freq: M, dtype: float64

In [21]:
wavelet = "bior3.3"
window = 60
representation = "SWT"
level = 1
data = rolling_window_image(pd.concat([train_tf, pd.Series([0,0,0,0,0,0,0,0,0,0,0,0], index=test.index)]), window, representation, wavelet, level)
X_train, X_test, y_train, _ = train_test_split(data, horizon)
rg = RidgeCV(alphas=np.logspace(-3, 3, 10))
rg.fit(X_train, y_train)
    
predictions = recursive_multistep_forecasting(X_test, rg, horizon)
preds = pd.Series(predictions, index=test.index)
preds_mexh = reverse_regressors(train, preds, format=transform)
preds_mexh

timestamp
2023-03    887581.700184
2023-04    857943.807993
2023-05    859519.433449
2023-06    821838.063375
2023-07    836419.594585
2023-08    854595.076466
2023-09    815099.322180
2023-10    849033.304975
2023-11    854416.916856
2023-12    990938.779945
2024-01    811771.752882
2024-02    819180.294710
Freq: M, dtype: float64

In [15]:
# data = rolling_window_image(pd.concat([train_tf, pd.Series([0,0,0,0,0,0,0,0,0,0,0,0], index=test.index)]), window, representation)
# X_train, X_test, y_train, _ = train_test_split(data, horizon)

# minirocket = Rocket(num_kernels=1000)  # by default, MiniRocket uses ~10_000 kernels
# minirocket.fit(X_train)

# X_train_transform = minirocket.transform(X_train)

# # classifier = RidgeCV(alphas=np.logspace(-3, 3, 10))
# classifier = RandomForestRegressor()
# classifier.fit(pd.DataFrame(X_train_transform), y_train)

# X_test_transform = minirocket.transform(X_test)

# predictions = recursive_multistep_rocket(X_test, minirocket, classifier, horizon)
# preds = pd.Series(predictions, index=test.index)
# preds_rocket = reverse_regressors(train, preds, format=transform)

In [16]:
# # plot_series(test, preds_deseasonal, preds_rf,preds_rocket, labels=["Test", "CWT- 36 - RIDGE","CWT- 36 - RF", "CWT - 36 - ROCKET - RIDGE"], title=f"{derivado} em {estado}")
# plot_series(test,
#              preds_deseasonal, 
#             preds_mexh,
#             preds_morl,
#             labels=["Test", "CWT- 36 - RIDGE morl+mexh",
#                 "CWT- 36 - RIDGE mexh",
#                 "CWT- 36 - RIDGE morl"
#                     ], 
#                     title=f"{derivado} em {estado}")